In [10]:
import cv2
import numpy as np

# Abrir 
cam = cv2.VideoCapture(0)

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

frame_reference = cv2.resize(frame_reference, (0, 0), fx=1.15, fy=1.15)
frame_reference = cv2.flip(frame_reference, 1)
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (31, 31), 0)

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
TIMEOUT = 50  # Número de frames até considerarmos que a pessoa saiu
DETECTION_DELAY = 200  # Número de frames para considerar uma nova pessoa

# Inicializar o detector ORB
orb = cv2.ORB_create()

# Função para detectar e descrever as features do ROI
def get_features(roi):
    keypoints, descriptors = orb.detectAndCompute(roi, None)
    return keypoints, descriptors

# Inicializar o Brute Force Matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

while True:
    ret, frame = cam.read()

    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (0, 0), fx=1.15, fy=1.15)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (31, 31), 0)

    # Subtrair o frame de referência do frame atual
    frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)
    _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos pequenos e criar uma lista de áreas
    valid_contours = []
    areas = [cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) >= 500]

    if areas:
        max_area = max(areas)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area >= 500 and area > max_area * 0.3:
                valid_contours.append(contour)

    # Loop sobre os contornos filtrados
    for contour in valid_contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        roi_gray = frame_gray[y:y+h, x:x+w]

        # Extrair keypoints e descritores das features
        keypoints, descriptors = get_features(roi_gray)

        if descriptors is not None:
            new_person = True

            for person in people_detected:
                # Comparar as features usando BFMatcher
                matches = bf.match(person['descriptors'], descriptors)
                matches = sorted(matches, key=lambda x: x.distance)

                # Se houver um número suficiente de correspondências, consideramos a pessoa como já detectada
                if len(matches) > 10:  # Valor arbitrário, pode ser ajustado
                    person['last_seen'] = (x, y)
                    person['missed_frames'] = 0
                    person['detection_delay'] += 1
                    new_person = False
                    break

            if new_person:
                # Adiciona nova pessoa após DETECTION_DELAY frames
                if person_id < DETECTION_DELAY:
                    person_id += 1
                    people_detected.append({
                        'id': person_id,
                        'descriptors': descriptors,
                        'last_seen': (x, y),
                        'missed_frames': 0,
                        'detection_delay': 1  # Inicia o contador para nova detecção
                    })
                    cv2.putText(frame, f"Pessoa {person_id} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "Esperando por nova detecção...", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Desenhar a casca convexa ao redor da pessoa detectada
        hull = cv2.convexHull(contour)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

    # Atualiza o contador de frames perdidos para todas as pessoas
    for person in people_detected:
        if person['missed_frames'] > TIMEOUT:
            people_detected.remove(person)
        else:
            person['missed_frames'] += 1

    # Mostrar IDs de todas as pessoas detectadas recentemente
    for person in people_detected:
        if person['missed_frames'] <= TIMEOUT:
            cv2.putText(frame, f"Pessoa {person['id']}", person['last_seen'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar o frame com as detecções
    cv2.imshow('Deteccao de Mudancas', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('Frame Delta', frame_delta)

    # Pressione 'q' para sair
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()
